In [5]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division

/home/lindb/anaconda2/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/home/lindb/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/lindb/anaconda2/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/home/lindb/anaconda2/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/home/lindb/anaconda2/lib/python2.7/site-packages/scipy/interpolate/_bsplines.py:10: RuntimeWarning: numpy.dtype size changed, may 

## download

In [9]:
# create --post-file for download
DIR = '/home/lindb/scratch/testdata/readSetLinks'
tmpfile = op.join(DIR,'.tmp.dat')
if not op.exists(tmpfile):
    with open(tmpfile,'w') as o:
        o.write("%s" % 'j_username=mlu1&j_password=rrgmmngu')
    !chmod 600 $tmpfile

In [6]:
# create dirs, open file with weblinks to fastq files
shdir = op.join(DIR,'shfiles')
if not op.exists(shdir):
    os.makedirs(shdir)
links = open(op.join(DIR,'readSetLinks.txt'),'r').readlines()
print len(links)
# make sure file name parse is correct
links[0].split("/")[-1].replace("\n","")

384


'HI.4779.005.D704---D504.LP_mg10_cap2_kit1_R2.fastq.gz'

In [16]:
# wget links
files = [f for f in fs('/home/lindb/scratch/testdata') if 'fastq' in f]
for i,link in enumerate(links):
    link = link.replace("\n","")
    fqgz = link.split("/")[-1]
    f = op.join('/home/lindb/scratch/testdata',fqgz)
    if not f in files: # faster to check if f in list than op.exists(f)
        text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=link%s
#SBATCH --export=all
#SBATCH --time=00:59:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1

cd /home/lindb/scratch/testdata
wget --no-cookies --no-check-certificate -c %s --post-file %s
''' % (str(i).zfill(3),
       link,
       tmpfile
      )
#         filE = op.join(shdir,'wget%s.sh' % str(i).zfill(3))
        filE = op.join(shdir,'wgetfinish%s.sh' % str(i).zfill(3))        
        with open(filE,'w') as o:
            o.write("%s" % text)

In [2]:
DIR = '/home/lindb/scratch/testdata/readSetLinks'
shdir = op.join(DIR,'shfiles')
fs(shdir)[0]

'/home/lindb/scratch/testdata/readSetLinks/shfiles/wget000.sh'

In [3]:
# sbatch with master
text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=wgetmaster
#SBATCH --export=all
#SBATCH --time=00:59:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1


'''
for f in fs(shdir):
    newtext = '''sbatch %s
''' % f
    text = text + newtext
filE=op.join(shdir,'master.sh')
with open(filE,'w') as o:
    o.write("%s" % text)

## check md5

In [20]:
# log to logfile
text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=md5sum
#SBATCH --export=all
#SBATCH --time=03:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1


cd /home/lindb/scratch/testdata/
md5sum -c *md5 > md5log.txt
'''
filE = op.join(shdir,'md5sumcheck.sh')
print filE
with open(filE,'w') as o:
    o.write("%s" % text)
!sbatch $fiLE

/home/lindb/scratch/testdata/readSetLinks/shfiles/md5sumcheck.sh
^C


In [7]:
# read the log, look for bad md5 checks
md5 = open('/home/lindb/scratch/testdata/md5log.txt').readlines()
for line in md5:
    res = line.split(" ")[1].split("\n")[0]
    if not res == "OK":
        print line
# one res in testdata was not ok, but I had gunzip-ed it

HI.4779.007.D704---D507.JP_101_2_cap3_kit1_R1.fastq.gz: FAILED open or read



## split files by library

In [8]:
from Bio import SeqIO

In [14]:
# get a list of fastq R1 files
files = sorted([f for f in fs('/scratch/lindb/testdata') if 'R1' in f and f.endswith('fastq.gz')])
len(files)

96

In [10]:
# import info sheet
summary = pd.read_csv('/scratch/lindb/testdata/1_CoAdapTree_Seq_summary.txt',sep='\t')
summary.head()

,Read_group_header_line,Pool_name,Sample_name,Ploidy,Index1,Index2,Adaptor_1,Adaptor_2
0,@RG\tID:mg8\tLB:LPcap2\tSM:mg8,LP_mg_cap2_kit1,LP_mg8_cap2_kit1,1,ATTACTCG,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
1,@RG\tID:mg22\tLB:LPcap2\tSM:mg22,LP_mg_cap2_kit1,LP_mg22_cap2_kit1,1,TCCGGAG,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
2,@RG\tID:mg64\tLB:LPcap2\tSM:mg64,LP_mg_cap2_kit1,LP_mg64_cap2_kit1,1,GAATTCGT,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
3,@RG\tID:mg9\tLB:LPcap2\tSM:mg9,LP_mg_cap2_kit1,LP_mg9_cap2_kit1,1,CGCTCATT,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
4,@RG\tID:mg10\tLB:LPcap2\tSM:mg10,LP_mg_cap2_kit1,LP_mg10_cap2_kit1,1,GAGATTCC,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT


In [11]:
samps = sorted(summary['Sample_name'].tolist())
len(samps)

96

In [39]:
# fill in blank pool names, create dictionary for samps in pools, get a list of spp, get rg info
sampdict = {}
spp      = []
rg       = {}
for row in summary.index:
    if type(summary.loc[row,'Pool_name']) == float:
        summary.loc[row,'Pool_name'] = summary.loc[row,'Sample_name']
    pool = summary.loc[row,'Pool_name']
    samp = summary.loc[row,'Sample_name']
    sp   = pool.split("_")[0]
    if not sp in spp:
        spp.append(sp)
        spp = sorted(spp)
    sampdict[samp] = pool
    rg[samp]       = summary.loc[row,'Read_group_header_line']


In [40]:
rg[samp]

'@RG\\tID:p27\\tLB:DFcap2\\tSM:p27'

In [43]:
pkldump(rg,'/scratch/lindb/testdata/rginfo.pkl')

In [44]:
# list of files by pool
poolfdict = {}
ftopool = {}
for f in files:
    samp = f.split("---")[1].split(".")[1].replace("_R1","")
    pool = sampdict[samp]
    if not pool in poolfdict:
        poolfdict[pool] = []
    f2 = op.abspath(f).replace("_R1","_R2")
    assert op.exists(f2)
    [poolfdict[pool].append(x) for x in [f,f2]]
    ftopool[f]  = pool
    ftopool[f2] = pool
    
# poolfdict

In [17]:
samp = sampdict.keys()[0]
sampdict[samp]

'JP_cap4_kit1'

## parse fastq files for efficiency

In [3]:
# multiprocess count files 
rc = Client(profile="job_10804682")
dview = rc[:]
lview = rc.load_balanced_view()
len(lview)

200

In [57]:
def countreads(f):
    linecount = !zcat $f | wc -l | bc
    readcount = int(linecount[0]) / 4
    return (f,readcount)
dview['countreads'] = countreads

In [103]:
allfiles = []
for f in files:
    f2 = f.replace("_R1","_R2")
    assert op.exists(f2)
    [allfiles.append(x) for x in [f,f2]]
counts = dview.map(countreads,allfiles)
len(counts)

192

In [129]:
poolreads = {}
freads = {}
for c in counts:
    f = c[0]
    freads[f] = c[1]
    pool = ftopool[f]
    if not pool in poolreads:
        poolreads[pool] = 0
    poolreads[pool] += c[1]
poolreads

{'DF_cap3_kit1': 133352664.0,
 'DF_cap4_kit1': 127930938.0,
 'DF_cap5_kit1': 158503122.0,
 'DF_p27_cap2_kit2': 154428352.0,
 'DF_p52_cap1_kit2': 109897366.0,
 'JP_cap3_kit1': 125895818.0,
 'JP_cap4_kit1': 127292702.0,
 'JP_cap5_kit1': 139053014.0,
 'JP_p101_cap1_kit1': 152988120.0,
 'JP_p102_cap2_kit1': 128065604.0,
 'LP_mg_cap2_kit1': 133595514.0,
 'LP_mg_cap3_kit1': 122122024.0,
 'LP_mg_cap4_kit1': 97997102.0,
 'LP_mg_cap5_kit1': 89972752.0,
 'LP_mg_cap6_kit1': 104349450.0,
 'LP_pmg40_cap1_kit1': 126757062.0,
 'WL_cap3_kit1': 109160064.0,
 'WL_cap4_kit1': 107461138.0,
 'WL_cap5_kit1': 160162326.0,
 'WL_p404_cap1_kit1': 141502964.0,
 'WL_p406_cap2_kit1': 158937502.0}

In [98]:
readsums = 0
for pool in sorted(poolreads,key=poolreads.get,reverse=True):
    print pool,poolreads[pool]
    readsums += poolreads[pool]
readsums

WL_cap5_kit1 80081163.0
WL_p406_cap2_kit1 79468751.0
DF_cap5_kit1 79251561.0
DF_p27_cap2_kit2 77214176.0
JP_p101_cap1_kit1 76494060.0
WL_p404_cap1_kit1 70751482.0
JP_cap5_kit1 69526507.0
LP_mg_cap2_kit1 66797757.0
DF_cap3_kit1 66676332.0
JP_p102_cap2_kit1 64032802.0
DF_cap4_kit1 63965469.0
JP_cap4_kit1 63646351.0
LP_pmg40_cap1_kit1 63378531.0
JP_cap3_kit1 62947909.0
LP_mg_cap3_kit1 61061012.0
DF_p52_cap1_kit2 54948683.0
WL_cap3_kit1 54580032.0
WL_cap4_kit1 53730569.0
LP_mg_cap6_kit1 52174725.0
LP_mg_cap4_kit1 48998551.0
LP_mg_cap5_kit1 44986376.0


1354712799.0

In [252]:
DIR = '/scratch/lindb/testdata/'
shdir = op.join(DIR,'shfiles/parse')
if not op.exists(shdir):
    os.makedirs(shdir)
count = 0
for f in ftopool.keys():
    pool = ftopool[f]
    pooldir = op.join(DIR,pool)
    if not op.exists(pooldir):
        os.makedirs(pooldir)
    text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=parse%s
#SBATCH --export=all
#SBATCH --time=08:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1

source $HOME/.bashrc
cd $HOME/pipeline

python 00_parse_fastq.py %s %s
''' % (str(count).zfill(3),
       f,
       pooldir
      )
    filE = op.join(shdir,'parse_%s.sh' % str(count).zfill(3))
    with open(filE,'w') as o:
        o.write("%s" % text)
    count += 1

In [171]:
os.chdir(shdir)
for f in fs(shdir):
    !sbatch $f

Submitted batch job 10807029
Submitted batch job 10807030
Submitted batch job 10807032
Submitted batch job 10807033
Submitted batch job 10807034
Submitted batch job 10807035
Submitted batch job 10807036
Submitted batch job 10807037
Submitted batch job 10807038
Submitted batch job 10807039
Submitted batch job 10807040
Submitted batch job 10807041
Submitted batch job 10807042
Submitted batch job 10807043
Submitted batch job 10807044
Submitted batch job 10807045
Submitted batch job 10807046
Submitted batch job 10807047
Submitted batch job 10807048
Submitted batch job 10807049
Submitted batch job 10807050
Submitted batch job 10807051
Submitted batch job 10807052
Submitted batch job 10807053
Submitted batch job 10807054
Submitted batch job 10807055
Submitted batch job 10807056
Submitted batch job 10807057
Submitted batch job 10807058
Submitted batch job 10807059
Submitted batch job 10807060
Submitted batch job 10807061
Submitted batch job 10807062
Submitted batch job 10807063
Submitted batc

In [175]:
for p in poolreads:
    print pool
    pdir = op.join(DIR,p)
    print len(ls(pdir))

DF_cap3_kit1
1089
DF_cap3_kit1
6643
DF_cap3_kit1
6230
DF_cap3_kit1
6002
DF_cap3_kit1
1678
DF_cap3_kit1
1029
DF_cap3_kit1
6390
DF_cap3_kit1
7440
DF_cap3_kit1
5014
DF_cap3_kit1
4038
DF_cap3_kit1
4067
DF_cap3_kit1
5476
DF_cap3_kit1
4164
DF_cap3_kit1
1159
DF_cap3_kit1
805
DF_cap3_kit1
1784
DF_cap3_kit1
6461
DF_cap3_kit1
902
DF_cap3_kit1
6339
DF_cap3_kit1
7519
DF_cap3_kit1
5801


## count files to decide lensh for 01a_trim-fastq

In [ ]:
1+1

In [3]:
# get nfiles per dir
os.chdir('/scratch/lindb/testdata/')
dirs = !ls -d */
dirs = [op.abspath(d) for d in dirs if 'kit' in d]

fcount = 0
dcount = {}
for d in dirs:
    fcount += len(ls(d))
    dcount[d] = len(ls(d))
fcount

271036

In [4]:
# determine frac of sh files
sums = sum(dcount.values())
frac = {}
for d in dcount:
    frac[d] = dcount[d]/sums
sum(frac.values()),frac

(0.9999999999999999,
 {'/scratch/lindb/testdata/DF_cap3_kit1': 0.04921855399282752,
  '/scratch/lindb/testdata/DF_cap4_kit1': 0.0472188196401954,
  '/scratch/lindb/testdata/DF_cap5_kit1': 0.058501453681429776,
  '/scratch/lindb/testdata/DF_p27_cap2_kit2': 0.05698136040968727,
  '/scratch/lindb/testdata/DF_p52_cap1_kit2': 0.040548119069053555,
  '/scratch/lindb/testdata/JP_cap3_kit1': 0.046466152097876294,
  '/scratch/lindb/testdata/JP_cap4_kit1': 0.046997446833630954,
  '/scratch/lindb/testdata/JP_cap5_kit1': 0.05132897474874187,
  '/scratch/lindb/testdata/JP_p101_cap1_kit1': 0.05645006567393261,
  '/scratch/lindb/testdata/JP_p102_cap2_kit1': 0.04725571510795614,
  '/scratch/lindb/testdata/LP_mg_cap2_kit1': 0.0493071031154533,
  '/scratch/lindb/testdata/LP_mg_cap3_kit1': 0.045078882510072464,
  '/scratch/lindb/testdata/LP_mg_cap4_kit1': 0.03617969568618191,
  '/scratch/lindb/testdata/LP_mg_cap5_kit1': 0.03322067917177054,
  '/scratch/lindb/testdata/LP_mg_cap6_kit1': 0.03852624743576499

In [5]:
# num sh files per dir
shfrac = {}
for d in frac:
    shfrac[d] = int(math.ceil(frac[d]*975))
    print d, shfrac[d]

/scratch/lindb/testdata/LP_mg_cap5_kit1 33
/scratch/lindb/testdata/WL_cap5_kit1 58
/scratch/lindb/testdata/WL_p406_cap2_kit1 58
/scratch/lindb/testdata/WL_cap3_kit1 40
/scratch/lindb/testdata/WL_cap4_kit1 39
/scratch/lindb/testdata/LP_mg_cap4_kit1 36
/scratch/lindb/testdata/JP_cap3_kit1 46
/scratch/lindb/testdata/LP_mg_cap6_kit1 38
/scratch/lindb/testdata/LP_mg_cap3_kit1 44
/scratch/lindb/testdata/LP_mg_cap2_kit1 49
/scratch/lindb/testdata/LP_pmg40_cap1_kit1 46
/scratch/lindb/testdata/WL_p404_cap1_kit1 51
/scratch/lindb/testdata/DF_cap4_kit1 47
/scratch/lindb/testdata/JP_p102_cap2_kit1 47
/scratch/lindb/testdata/DF_cap5_kit1 58
/scratch/lindb/testdata/JP_p101_cap1_kit1 56
/scratch/lindb/testdata/JP_cap4_kit1 46
/scratch/lindb/testdata/DF_p27_cap2_kit2 56
/scratch/lindb/testdata/JP_cap5_kit1 51
/scratch/lindb/testdata/DF_p52_cap1_kit2 40
/scratch/lindb/testdata/DF_cap3_kit1 48


In [46]:
shdir = '/scratch/lindb/testdata/shfiles/start_trim'
if not op.exists(shdir):
    os.makedirs(shdir)
for d in shfrac:
    text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=%s
#SBATCH --export=all
#SBATCH --time=12:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1
#SBATCH --mail-user=brandon.lind@ubc.ca
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL


source $HOME/.bashrc
cd $HOME/pipeline

python 01a_trim-fastq.py %s %s %s
''' % (op.basename(d),
       d,
       '/home/lindb/scratch/ptaeda.v1.01.reduced.pseudo.fasta',
       str(shfrac[d])
      )
    filE = op.join(shdir,'start-trim_%s.sh' % op.basename(d))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [48]:
for f in fs(shdir):
    !sbatch $f

Submitted batch job 10822585
Submitted batch job 10822587
Submitted batch job 10822590
Submitted batch job 10822592
Submitted batch job 10822594
Submitted batch job 10822597
Submitted batch job 10822599
Submitted batch job 10822603
Submitted batch job 10822606
Submitted batch job 10822608
Submitted batch job 10822612
Submitted batch job 10822614
Submitted batch job 10822617
Submitted batch job 10822619
Submitted batch job 10822621
Submitted batch job 10822623
Submitted batch job 10822628
Submitted batch job 10822631
Submitted batch job 10822633
Submitted batch job 10822635
Submitted batch job 10822638


In [50]:
os.getcwd()

'/scratch/lindb/testdata'

In [1]:
text = !squeue -u lindb

In [2]:
count = 0
pids = []
for line in text[1:]:
#     print line.split(" ")[0]
    pids.append(line.split(" ")[0])
pids

['10823132',
 '10823133',
 '10823135',
 '10823136',
 '10823138',
 '10823140',
 '10823142',
 '10823143',
 '10823144',
 '10823145',
 '10823146',
 '10823148',
 '10823149',
 '10823150',
 '10823151',
 '10823152',
 '10823153',
 '10823155',
 '10823158',
 '10823159',
 '10823162',
 '10823164',
 '10823165',
 '10823167',
 '10823169',
 '10823170',
 '10823174',
 '10823176',
 '10823178',
 '10823179',
 '10823180',
 '10823183',
 '10823186',
 '10823189',
 '10823191',
 '10823193',
 '10823195',
 '10823196',
 '10823198',
 '10823200',
 '10823201',
 '10823207',
 '10823208',
 '10823210',
 '10823211',
 '10823214',
 '10823216',
 '10823217',
 '10823218',
 '10823221',
 '10823222',
 '10823224',
 '10823228',
 '10823231',
 '10823232',
 '10823235',
 '10823236',
 '10823237',
 '10823239',
 '10823240',
 '10823245',
 '10823246',
 '10823249',
 '10823258',
 '10823259',
 '10823260',
 '10823263',
 '10823266',
 '10823267',
 '10823269',
 '10823270',
 '10823271',
 '10823273',
 '10823276',
 '10823277',
 '10823279',
 '10823281',

In [4]:
filE = '/home/lindb/scancel.txt'
with open(filE,'w') as o:
    for p in pids:
        p = int(p)
        o.write("scancel %i\n" % p)

In [ ]:
!cat /home/lindb/scancel.txt | parallel